## Data cleaning

### Set up

We first import the different libraries that we will be using for this project

In [448]:
import pandas as pd
from unidecode import unidecode
import numpy as np

We import our raw dataset

In [449]:
data_path="data/fires-all.csv"
try:
    fires=pd.read_csv(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

,id,superficie,fecha,lat,lng,latlng_explicit,idcomunidad,idprovincia,idmunicipio,municipio,...,causa_supuesta,causa_desc,muertos,heridos,time_ctrl,time_ext,personal,medios,gastos,perdidas
0,1968290001,14.0,1968-01-01,NaN,NaN,0,4,29,0,INDETERMINADO,...,1,40,0,0,0,360,0,0,0,0
1,1968430003,3.0,1968-01-03,NaN,NaN,0,2,43,0,INDETERMINADO,...,1,0,0,0,0,60,0,0,0,0
2,1968290006,2.0,1968-01-06,NaN,NaN,0,4,29,0,INDETERMINADO,...,1,0,0,0,0,120,0,0,0,0
3,1968430016,600.0,1968-01-07,NaN,NaN,0,2,43,0,INDETERMINADO,...,1,20,0,0,0,1440,35,1,0,0
4,1968120007,8.2,1968-01-07,NaN,NaN,0,9,12,0,INDETERMINADO,...,1,20,0,0,0,120,0,0,0,0


We analyze the data and observe the type of data on each column and how many nulls values we have

In [450]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284589 entries, 0 to 284588
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               284589 non-null  int64  
 1   superficie       284589 non-null  float64
 2   fecha            284589 non-null  object 
 3   lat              230154 non-null  float64
 4   lng              230154 non-null  float64
 5   latlng_explicit  284589 non-null  int64  
 6   idcomunidad      284589 non-null  int64  
 7   idprovincia      284589 non-null  int64  
 8   idmunicipio      284589 non-null  int64  
 9   municipio        284576 non-null  object 
 10  causa            284589 non-null  int64  
 11  causa_supuesta   284589 non-null  int64  
 12  causa_desc       284589 non-null  int64  
 13  muertos          284589 non-null  int64  
 14  heridos          284589 non-null  int64  
 15  time_ctrl        284589 non-null  int64  
 16  time_ext         284589 non-null  int6

### Remove unnecesary columns

In [451]:
#drop
try:
    fires.drop(["id","causa","causa_supuesta","causa_desc",
                "muertos","heridos","time_ctrl","time_ext",
                "personal","medios","gastos","perdidas",
                "latlng_explicit"],axis=1, inplace=True)
except Exception as error:
    print(f"ERROR while droping the columns {error}")
fires.head()

,superficie,fecha,lat,lng,idcomunidad,idprovincia,idmunicipio,municipio
0,14.0,1968-01-01,NaN,NaN,4,29,0,INDETERMINADO
1,3.0,1968-01-03,NaN,NaN,2,43,0,INDETERMINADO
2,2.0,1968-01-06,NaN,NaN,4,29,0,INDETERMINADO
3,600.0,1968-01-07,NaN,NaN,2,43,0,INDETERMINADO
4,8.2,1968-01-07,NaN,NaN,9,12,0,INDETERMINADO


### Remove unnecesary rows

In [452]:
#We only want the fires of the comunidad 3 (Galicia)
fires=fires[fires["idcomunidad"]==3]
#We drop null values on lat and lng, beacsue we cannot find where the fire happend,
#  and it is older data from  1968
fires=fires.dropna(subset=["lat","lng"])
#We remove the column idcomunidad as it is no longer need it
fires.drop(["idcomunidad"],axis=1,inplace=True)
#print how many null values are for each column
fires.isna().sum(),
#TODO: from the coordinates lat and lng obtain the blanks municipio

(superficie     0
 fecha          0
 lat            0
 lng            0
 idprovincia    0
 idmunicipio    0
 municipio      5
 dtype: int64,)

In [453]:
fires.head()

,superficie,fecha,lat,lng,idprovincia,idmunicipio,municipio
39703,20.0,1980-09-18,42.428281,-6.914337,32,9,"BARCO DE VALDEORRAS, O"
54407,2.0,1983-01-16,42.542185,-8.449205,36,12,COTOBADE
54410,6.0,1983-01-16,42.102572,-8.415920,36,34,"NEVES, AS"
54415,3.0,1983-01-17,43.629834,-7.367642,27,19,FOZ
54417,40.0,1983-01-18,43.018968,-7.408954,27,11,CASTROVERDE


In [454]:
#We save the data for the time series prediction
data_path="data/fires-time-series.csv"
try:
    fires.to_csv(data_path,index=False)
except Exception as error:
    print(f"Error while exporting the data to the excel file: {error}")

In [455]:
data_path="data/fires-time-series.csv"
try:
    fires_time_series=pd.read_csv(data_path)
except Exception as error:
    print(f"Error while importing the excel file: {error}")
fires.head()

,superficie,fecha,lat,lng,idprovincia,idmunicipio,municipio
39703,20.0,1980-09-18,42.428281,-6.914337,32,9,"BARCO DE VALDEORRAS, O"
54407,2.0,1983-01-16,42.542185,-8.449205,36,12,COTOBADE
54410,6.0,1983-01-16,42.102572,-8.415920,36,34,"NEVES, AS"
54415,3.0,1983-01-17,43.629834,-7.367642,27,19,FOZ
54417,40.0,1983-01-18,43.018968,-7.408954,27,11,CASTROVERDE


### Añadir datos meteologicos para la prediccion con variables exogenas

#### Limpiar datos de fires.csv

Se tendrá en cuenta los datos de incendios despues de 2009, ya que Solo tenemos informacion valido a partir del 2009 de meteorologia

In [456]:
#convertimos las fechas a tipo datetime (para una mejor compatibilidad)
fires['fecha'] = pd.to_datetime(fires['fecha'], errors='coerce')
#filtrar fechas que empiezen a partir del 2005 hasta el 2018
fires = fires.loc[fires['fecha'] >= '2009-01-01']
fires.head()

,superficie,fecha,lat,lng,idprovincia,idmunicipio,municipio
244077,1.0,2009-01-11,42.314177,-8.198452,32,18,CARBALLEDA DE AVIA
244079,5.0,2009-01-11,41.996375,-7.170321,32,34,"GUDIÑA, A"
244082,2.7,2009-01-11,42.333021,-8.314144,36,13,COVELO
244090,2.0,2009-01-12,41.995624,-7.174073,32,34,"GUDIÑA, A"
244094,11.1,2009-01-12,42.290851,-7.620661,32,43,MACEDA


Obtener el código de cada provincia y ver la cantidad de datos de cada uno de ellos

In [457]:
valores_unicos = fires['idprovincia'].unique()
print(valores_unicos)
for valor in valores_unicos:
    print((fires['idprovincia'] == valor).sum())

[32 36 27 15]
4221
1349
1159
1608


La comunidad de Ourense es la que más datos presenta

In [458]:
fires[fires['idprovincia']==32].head()

,superficie,fecha,lat,lng,idprovincia,idmunicipio,municipio
244077,1.0,2009-01-11,42.314177,-8.198452,32,18,CARBALLEDA DE AVIA
244079,5.0,2009-01-11,41.996375,-7.170321,32,34,"GUDIÑA, A"
244090,2.0,2009-01-12,41.995624,-7.174073,32,34,"GUDIÑA, A"
244094,11.1,2009-01-12,42.290851,-7.620661,32,43,MACEDA
244227,1.5,2009-02-15,42.005325,-7.009491,32,48,"MEZQUITA, A"


Filtrar los datos de incendios a Ourense

In [459]:
#Juntar por semanas en la provincia de Ourense (id32)
# Convertir 'fecha' a datetime
fires['fecha'] = pd.to_datetime(fires['fecha'])

# Filtrar por la provincia 32
fires = fires[fires['idprovincia'] == 32]

fires.drop(["lat","lng","idmunicipio","municipio","idprovincia"],axis=1,inplace=True)

fires.head()

,superficie,fecha
244077,1.0,2009-01-11
244079,5.0,2009-01-11
244090,2.0,2009-01-12
244094,11.1,2009-01-12
244227,1.5,2009-02-15


Crear una nueva columna de ano y semana y agrupar los datos de superficie y numero de incendio

In [460]:
# Convertir 'fecha' a formato datetime
fires["fecha"] = pd.to_datetime(fires["fecha"], format="%Y-%m-%d")

# Crear nuevas columnas para el año y la semana
fires['Anno'] = fires['fecha'].dt.year
fires['Semana'] = fires['fecha'].dt.isocalendar().week

fires = fires.groupby(["Anno", "Semana"]).agg({
    'superficie': 'sum',           # Suma de superficie
    'fecha': 'count'               # Cuenta de filas agrupadas (número de incendios)
}).rename(columns={'fecha': 'numero_incendios'}).round(2)

fires= fires.reset_index()
fires.head()

,Anno,Semana,superficie,numero_incendios
0,2009,2,6.00,2
1,2009,3,13.10,2
2,2009,7,1.50,1
3,2009,8,96.85,25
4,2009,9,227.66,64


In [461]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Anno              342 non-null    int32  
 1   Semana            342 non-null    UInt32 
 2   superficie        342 non-null    float64
 3   numero_incendios  342 non-null    int64  
dtypes: UInt32(1), float64(1), int32(1), int64(1)
memory usage: 8.5 KB


#### Juntamos todos los datos de cada año en un unico csv por cada estacion meteo


In [462]:
#Definimos las estaciones de meteorologia
stations=["C01_A Capela_","C02_Boimorto_","LU01_Castro de Rei_","LU02_Monforte de Lemos_"]
station_data = {station: pd.DataFrame() for station in stations}

#Iteramos por todos los años que tenemos disponibles de datos meteorologicos
for year in range(2009,2019):
    data_path=f"data/{year}_{year+1}/"
    # Iterar sobre cada esación
    for station in stations:
        try:
            # Situarlo en el directorio y abrirlo
            station_path=data_path+f"{station}01_01_{year}_01_01_{year+1}.csv"
            aux = pd.read_csv(station_path, encoding="utf-16")

            # normalizar los caracteres y filtar solo los datos relevantes 
            aux.columns = [unidecode(col) for col in aux.columns]
            aux = aux[["Fecha", "Temp Media (oC)", "Temp Max (oC)", "Temp Minima (oC)" , "Humedad Media (%)", "Humedad Max (%)", "Humedad Min (%)", "Velviento (m/s)","DirViento (o)", "VelVientoMax (m/s)", "Radiacion (MJ/m2)", "Precipitacion (mm)"]]

            # Guardar los datos de la estacion junto con los de otros años. 
            station_data[station]=pd.concat([station_data[station],aux],ignore_index=True)
        except pd.errors.ParserError as parse_error:
            print(f"[ERROR]: Parser error when reading {station_path}: {parse_error}")
        except FileNotFoundError as file_error:
            print(f"[ERROR]: File not found: {station_path}")
        except Exception as general_error:
            print(f"[ERROR]: General error occurred while reading the file: {general_error}")
    print(f"Datos cargados para del año {year}")



Datos cargados para del año 2009
Datos cargados para del año 2010
Datos cargados para del año 2011
Datos cargados para del año 2012
Datos cargados para del año 2013
Datos cargados para del año 2014
Datos cargados para del año 2015
Datos cargados para del año 2016
Datos cargados para del año 2017
Datos cargados para del año 2018


#### Imputacion
Como existen datos nulos en algunos campos, decidimos realizar una imputacion usando la media total de la columna 

In [463]:
aux = ["Temp Media (oC)", "Temp Max (oC)", "Temp Minima (oC)" , "Humedad Media (%)", "Humedad Max (%)", "Humedad Min (%)", "Velviento (m/s)","DirViento (o)", "VelVientoMax (m/s)", "Radiacion (MJ/m2)", "Precipitacion (mm)"]
#  Iteramos sobre cada DataFrame en el diccionario `station_data`
for aux_value in aux :
    for station, data in station_data.items():
        try:
            if not data.empty:
                
                 # Reemplazamos valores vacíos ("" o None) con NaN
                data.replace(["", None], pd.NA, inplace=True)
                if aux_value in data.columns:
                    # Calculamos la media ignorando los valores NaN
                    column_mean = data[aux_value].mean(skipna=True).round(4)

                    # Imputamos los valores vacíos (NaN) con la media calculada
                    data[aux_value] = data[aux_value].fillna(column_mean)

                    # Guardamos el DataFrame modificado de vuelta en el diccionario
                    station_data[station] = data

            else:
                print(f"El DataFrame de la estación {station} está vacío.")

        except Exception as error:
            print(f"[ERROR]: Error al procesar la estación {station}: {error}")

print("Imputación completada.")

Imputación completada.


#### Agragacion de datos por año y semana
Como vamos a juntar los datos por año y semana, sería interesante obtener la media y la varianza de cada semana para tenerlo cuenta en nuestras predicciones.

Además, para cada fila, también incluiremos nuevas columnas de los mismos datos de la semana pasada. 

In [464]:
# Iteramos sobre cada estación y sus respectivos datos
for station, data in station_data.items():
    try:
        # Convertimos la columna 'Fecha' a formato datetime
        data["Fecha"] = pd.to_datetime(data["Fecha"], format="%d/%m/%Y")
                
        # Creamos nuevas columnas para el año y la semana
        data['Anno'] = data['Fecha'].dt.year
        data['Semana'] = data['Fecha'].dt.isocalendar().week
        
        # Agrupamos por año y semana y calculamos la media y suvarianza
        weekly_data = data.groupby(['Anno', 'Semana']).agg({
            "Temp Media (oC)": ['mean', 'var'],
            "Temp Max (oC)": ['mean', 'var'],
            "Temp Minima (oC)": ['mean', 'var'],
            "Humedad Media (%)": ['mean', 'var'],
            "Humedad Max (%)": ['mean', 'var'],
            "Humedad Min (%)": ['mean', 'var'],
            "Velviento (m/s)": ['mean', 'var'],
            "DirViento (o)": ['mean', 'var'],
            "VelVientoMax (m/s)": ['mean', 'var'],
            "Precipitacion (mm)": ['mean', 'var']
        }).round(4)

        # Aplanamos los nombres de columnas
        weekly_data.columns = ['_'.join(col).strip() for col in weekly_data.columns.values]

        # Reset index para tener un DataFrame estándar
        weekly_data = weekly_data.reset_index()
        
        # Añadir columnas de los datos de la semana anterior
        for column in weekly_data.columns:
                if column not in ['Anno', 'Semana']:
                    # Shift con manejo explícito del cruce de años
                    weekly_data[f'{column}_Semana_Pasada'] = weekly_data.groupby('Anno')[column].shift(1)
                    
                    # Si es la primera semana del año, toma la última semana del año anterior
                    first_week_indices = weekly_data['Semana'] == 1
                    prev_year_indices = weekly_data[first_week_indices].index
                    for idx in prev_year_indices:
                        current_year = weekly_data.loc[idx, 'Anno']
                        prev_year = current_year - 1
                        last_week_prev_year = weekly_data[(weekly_data['Anno'] == prev_year) & 
                                                        (weekly_data['Semana'] == weekly_data['Semana'].max())]
                        if not last_week_prev_year.empty:
                            weekly_data.loc[idx, f'{column}_Semana_Pasada'] = last_week_prev_year[column].values[0]


        # Guardar datos semanales en el diccionario
        station_data[station] = weekly_data

        #station_data[station]['Valor_Anterior'] = station_data[station]['Temp Media (oC)_mean'].shift(1)

        print(f"Procesamiento semanal completo para {station}")
    
    except Exception as error:
        print(f"[ERROR]: Error al procesar los datos semanales para {station}: {error}")


Procesamiento semanal completo para C01_A Capela_
Procesamiento semanal completo para C02_Boimorto_
Procesamiento semanal completo para LU01_Castro de Rei_
Procesamiento semanal completo para LU02_Monforte de Lemos_


Exportar los datos limpiados de cada estacion para un futuro uso 

In [465]:

# Ahora, para cada estación, concatenamos todos los DataFrames y los guardamos en un archivo CSV único
# Iteramos sobre cada estación y sus respectivos datos en el diccionario station_data
for station, data in station_data.items():
    try:
        # Definimos el nombre del archivo de salida
        output_filename = f"data/estaciones/{station}01_01_2009_01_01_2019.csv"
        
        # Guardamos los datos concatenados en un archivo CSV
        data.to_csv(output_filename, index=False, encoding="utf-16")
        print(f"Archivo guardado para {station}: {output_filename}")
    
    except Exception as error:
        # Manejo de errores al guardar los datos
        print(f"[ERROR]: Error al guardar los datos para {station}: {error}")


Archivo guardado para C01_A Capela_: data/estaciones/C01_A Capela_01_01_2009_01_01_2019.csv
Archivo guardado para C02_Boimorto_: data/estaciones/C02_Boimorto_01_01_2009_01_01_2019.csv
Archivo guardado para LU01_Castro de Rei_: data/estaciones/LU01_Castro de Rei_01_01_2009_01_01_2019.csv
Archivo guardado para LU02_Monforte de Lemos_: data/estaciones/LU02_Monforte de Lemos_01_01_2009_01_01_2019.csv


#### Selección de la provincia de Ourense
Como se ha visto previamente, ourense es la que más datos de incendio presenta, por lo que es interesante empezar nuestras preddiciones por esa semana.

Para ello, hemos agrupado tanto los datos de incendio y los datos de datos meteologicos en año y semana, por lo que con solo hacer un merge de ambos dataframe, obtendremos nuestra dataset final para realizar las preddciones con valores exógenos.

In [466]:

# Realizar un outer merge para obtener todos los datos
weather_fires_time_series = pd.merge(station_data['LU02_Monforte de Lemos_'], fires, on=['Anno', 'Semana'], how='outer')

# Los datos que de una semana que existe en un dataset y no en otro se rellenará con 0
weather_fires_time_series['superficie'] = weather_fires_time_series['superficie'].fillna(0)
weather_fires_time_series['numero_incendios'] = weather_fires_time_series['numero_incendios'].fillna(0)

# Filtar el primero datos
weather_fires_time_series = weather_fires_time_series[weather_fires_time_series['Anno'] < 2019].reset_index(drop=True)
weather_fires_time_series = weather_fires_time_series[weather_fires_time_series['Anno'] > 2008].reset_index(drop=True)
weather_fires_time_series = weather_fires_time_series[~((weather_fires_time_series['Anno'] == 2009) & ((weather_fires_time_series['Semana'] == 1)))]

N = 52

# Transformaciones cíclicas
weather_fires_time_series['Sin_Semana'] = np.sin(2 * np.pi * weather_fires_time_series['Semana'] / N)
weather_fires_time_series['Cos_Semana'] = np.cos(2 * np.pi * weather_fires_time_series['Semana'] / N)
weather_fires_time_series = weather_fires_time_series.drop(columns='Semana')

weather_fires_time_series['superficie_anterior'] = weather_fires_time_series['superficie'].shift(1)
weather_fires_time_series['numero_anterior'] = weather_fires_time_series['numero_incendios'].shift(1)


In [467]:
# #We save the data with the weather
# data_path="data/fires-weather_timeseries.csv"
# try:
#     weather_fires.to_csv(data_path,index=False)
# except Exception as error:
#     print(f"Error while exporting the data to the excel file: {error}")

In [468]:
def ajustar_semana(row):
    if row['Semana'] == 51:
        row['Semana'] = 52
        row['Anno'] -= 1
    else:
        row['Semana'] -= 1
    return row

# Aplicar la función al DataFrame
fires = fires.apply(ajustar_semana, axis=1)

# Realizar un outer merge para obtener todos los datos
weather_fires = pd.merge(station_data['LU02_Monforte de Lemos_'], fires, on=['Anno', 'Semana'], how='outer')

# Los datos que de una semana que existe en un dataset y no en otro se rellenará con 0
weather_fires['superficie'] = weather_fires['superficie'].fillna(0)
weather_fires['numero_incendios'] = weather_fires['numero_incendios'].fillna(0)

# Filtar el primero datos
weather_fires = weather_fires[weather_fires['Anno'] < 2019].reset_index(drop=True)
weather_fires = weather_fires[weather_fires['Anno'] > 2008].reset_index(drop=True)
weather_fires = weather_fires[~((weather_fires['Anno'] == 2009) & ((weather_fires['Semana'] == 1)))]

N = 52

# Transformaciones cíclicas
weather_fires['Sin_Semana'] = np.sin(2 * np.pi * weather_fires['Semana'] / N)
weather_fires['Cos_Semana'] = np.cos(2 * np.pi * weather_fires['Semana'] / N)
weather_fires = weather_fires.drop(columns='Semana')

weather_fires['superficie_actual'] = weather_fires['superficie'].shift(1)
weather_fires['numero_actual'] = weather_fires['numero_incendios'].shift(1)

weather_fires.rename(columns={"superficie": "superficie_siguiente"}, inplace=True)
weather_fires.rename(columns={"numero_incendios": "numero_incendios_siguiente"}, inplace=True)


In [469]:
weather_fires.head()

,Anno,Temp Media (oC)_mean,Temp Media (oC)_var,Temp Max (oC)_mean,Temp Max (oC)_var,Temp Minima (oC)_mean,Temp Minima (oC)_var,Humedad Media (%)_mean,Humedad Media (%)_var,Humedad Max (%)_mean,...,VelVientoMax (m/s)_mean_Semana_Pasada,VelVientoMax (m/s)_var_Semana_Pasada,Precipitacion (mm)_mean_Semana_Pasada,Precipitacion (mm)_var_Semana_Pasada,superficie_siguiente,numero_incendios_siguiente,Sin_Semana,Cos_Semana,superficie_actual,numero_actual
1,2009.0,-0.0100,5.6949,6.0243,1.2704,-4.7957,7.6270,84.9143,64.8848,97.2714,...,2.2250,0.4458,2.1500,3.2367,13.1,2.0,0.239316,0.970942,<NA>,<NA>
2,2009.0,5.2371,9.2971,10.8743,6.9375,0.2771,23.3576,83.9286,22.9157,97.4571,...,3.3657,2.5485,0.0286,0.0057,0.0,0.0,0.354605,0.935016,13.1,2.0
3,2009.0,7.0757,16.1478,10.7657,13.3214,3.2914,15.5250,86.3514,65.5492,95.9571,...,4.6986,2.2187,1.4857,1.4648,0.0,0.0,0.464723,0.885456,0.0,0.0
4,2009.0,8.4686,9.2649,11.7971,9.0843,4.6414,13.2162,82.7800,187.0814,97.6000,...,8.4100,11.9590,10.0571,65.8895,0.0,0.0,0.568065,0.822984,0.0,0.0
5,2009.0,3.5871,3.4337,8.7414,4.8479,-0.5229,4.2174,89.0714,8.9690,98.8857,...,6.6129,10.2418,9.3143,51.9448,1.5,1.0,0.663123,0.748511,0.0,0.0


In [470]:
weather_fires.info()

<class 'pandas.core.frame.DataFrame'>
Index: 529 entries, 1 to 529
Data columns (total 47 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Anno                                   529 non-null    Float64
 1   Temp Media (oC)_mean                   524 non-null    float64
 2   Temp Media (oC)_var                    524 non-null    float64
 3   Temp Max (oC)_mean                     524 non-null    float64
 4   Temp Max (oC)_var                      524 non-null    float64
 5   Temp Minima (oC)_mean                  524 non-null    float64
 6   Temp Minima (oC)_var                   524 non-null    float64
 7   Humedad Media (%)_mean                 524 non-null    float64
 8   Humedad Media (%)_var                  524 non-null    float64
 9   Humedad Max (%)_mean                   524 non-null    float64
 10  Humedad Max (%)_var                    524 non-null    float64
 11  Humedad Min

In [471]:
#We save the data with the weather
data_path="data/fires-weather.csv"
try:
    weather_fires.to_csv(data_path,index=False)
except Exception as error:
    print(f"Error while exporting the data to the excel file: {error}")